# Draw Focus for paper


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS

- creation date : May 12th 2020



In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib as mpl
import pandas as pd
import itertools
import matplotlib.gridspec as gridspec

In [2]:
# to enlarge the sizes
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (13, 13),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'font.size': 14}
plt.rcParams.update(params)

## Constants for conversions

In [3]:
m_to_mm=1000.
mm_to_m=1e-3
inch_to_mm=25.4
mm_to_inch=1./inch_to_mm
micr_to_m=1e-6
micr_to_mm=1e-3
m_to_micr=1./micr_to_m
m_to_cm=100.
m_to_nm=1e9
nm_to_m=1./m_to_nm

In [4]:
arcdeg_to_arcmin=60.
arcmin_to_arcdeg=1./arcdeg_to_arcmin
arcmin_to_arcsec=60.
arcdeg_to_arcsec=arcdeg_to_arcmin*arcmin_to_arcsec
arcsec_to_arcdeg=1./arcdeg_to_arcsec

In [5]:
deg_to_rad=np.pi/180.

In [6]:
rad_to_deg=1./deg_to_rad
rad_to_arcsec=rad_to_deg*arcdeg_to_arcsec
rad_to_arcmin=rad_to_deg*arcdeg_to_arcmin
arcmin_ro_rad=1./rad_to_arcmin

## Configuration parameters at the telescope

#### telescope

In [7]:
Tel_Focal_Length=12.6 # m : Focal length of the telescope
Tel_Diameter=0.9 # m : Diameter of the telescope    
Tel_Fnum=Tel_Focal_Length/Tel_Diameter

#### filter

In [8]:
Filt_D=0.05769 # m distance of the filter position wrt CCD plane
Filt_size=3*inch_to_mm

#### CCD detector

In [9]:
Det_xpic=24.0 # microns per pixel
#Det_NbPix=2048 # number of pixels per CCD side
Det_NbPix=2512 # number of pixels per CCD side For 400 only
Det_size=Det_xpic*Det_NbPix*micr_to_mm # CCD size in mm, 5 cm or 2 inch

## Input file

In [10]:
# number of rays
NBEAM_X=11
NBEAM_Y=11
NBEAM=NBEAM_X*NBEAM_Y
NWL=4
NBTOT=NBEAM*NWL

In [11]:
theta_x=0.  # angle in arcmin
theta_y=0.  # angle in arcmin 

In [12]:
theta_x_num=int(theta_x*10)
theta_y_num=int(theta_y*10)

In [13]:
if theta_x_num>0:
    theta_nstr='{:0>2}'.format(theta_x_num)
    theta_x_str="p"+theta_nstr
else:
    theta_nstr='{:0>2}'.format(-theta_x_num)
    theta_x_str="m"+theta_nstr

In [14]:
if theta_y_num>0:
    theta_nstr='{:0>2}'.format(theta_y_num)
    theta_y_str="p"+theta_nstr
else:
    theta_nstr='{:0>2}'.format(-theta_y_num)
    theta_y_str="m"+theta_nstr

In [15]:
!ls

Beam4_Rayfile_121_1000nm_m00_m00.xlsx DrawFocus.ipynb
Beam4_Rayfile_121_400nm_m00_m00.xlsx  HOE
Beam4_Rayfile_121_600nm_m00_m00.xlsx  Make_B4SquareBeam_OneWL.ipynb
Beam4_Rayfile_121_800nm_m00_m00.xlsx  R200
DrawBeams.ipynb                       R400


In [16]:
Beam4_Rayfile="Beam4_Rayfile_{:d}_allwl_{}_{}".format(NBTOT,theta_x_str,theta_y_str)

In [17]:
Beam4_Rayfile

'Beam4_Rayfile_484_allwl_m00_m00'

# Read input files

In [18]:
rayfile_hoe=Beam4_Rayfile+"_RAY_HOE_OUT.xlsx"
rayfile_R400=Beam4_Rayfile+"_RAY_RON400_OUT.xlsx"
rayfile_R200=Beam4_Rayfile+"_RAY_RON200_OUT.xlsx"

In [19]:
input_hoe = pd.ExcelFile(rayfile_hoe)
df_hoe = input_hoe.parse(index_row=0,header=1)
df_hoe=df_hoe.iloc[0:NBTOT]

FileNotFoundError: [Errno 2] No such file or directory: 'Beam4_Rayfile_484_allwl_m00_m00_RAY_HOE_OUT.xlsx'

In [ ]:
# rename columns otherwise they are not recognize
#df_hoe.columns = ["X0","Y0","Z0","U0","V0","W0","wave","X1","Y1","Z1","X2","Y2","Z2","X3","Y3","Z3","Xfinal","Yfinal","Zfinal","Notes"]
df_hoe.columns = ["Y0","X0","Z0","V0","U0","W0","wave","Y1","X1","Z1","Y2","X2","Z2","Y3","X3","Z3","Yfinal","Xfinal","Zfinal","Notes"]

In [ ]:
df_hoe

In [ ]:
input_R400 = pd.ExcelFile(rayfile_R400)
df_R400 = input_R400.parse(index_row=0,header=1)
df_R400=df_R400.iloc[0:NBTOT]

In [ ]:
# rename columns otherwise they are not recognize
df_R400.columns = ["X0","Y0","Z0","U0","V0","W0","wave","X1","Y1","Z1","X2","Y2","Z2","X3","Y3","Z3","Xfinal","Yfinal","Zfinal","Notes"]

In [ ]:
df_R400

In [ ]:
input_R200 = pd.ExcelFile(rayfile_R200)
df_R200 = input_R200.parse(index_row=0,header=1)
df_R200=df_R200.iloc[0:NBTOT]

In [ ]:
# rename columns otherwise they are not recognize
df_R200.columns = ["X0","Y0","Z0","U0","V0","W0","wave","X1","Y1","Z1","X2","Y2","Z2","X3","Y3","Z3","Xfinal","Yfinal","Zfinal","Notes"]

In [ ]:
df_R200

# Draw beam

In [ ]:
ColorDict = {0.0004:"b",0.0006:"g",0.0008:"r",0.001:"k"}

In [ ]:
df_R400['Z0']

In [ ]:
plt.figure(figsize=(16,8))
for index, row in df_R400.iterrows():
    wl=row["wave"]
    col=ColorDict[wl]
    X0=row["X0"];Y0=row["Y0"];Z0=row["Z0"]-115.5
    X1=row["X1"];Y1=row["Y1"];Z1=row["Z1"]-115.5
    X2=row["X2"];Y2=row["Y2"];Z2=row["Z2"]-115.5
    X3=row["X3"];Y3=row["Y3"];Z3=row["Z3"]-115.5
    if Y0==0:
        plt.plot([Z0,Z1],[X0,X1],"-",color=col)
        plt.plot([Z1,Z2],[X1,X2],"-",color=col)
        plt.plot([Z2,Z3],[X2,X3],"-",color=col)
plt.grid()
plt.xlim(-70,0)
plt.ylim(-10,30)
    

In [ ]:
# gridspec inside gridspec
fig = plt.figure(figsize=(16,16))
gs0 = gridspec.GridSpec(2, 1, figure=fig)

ax1 = fig.add_subplot(gs0[0])
for index, row in df_R400.iterrows():
    wl=row["wave"]
    col=ColorDict[wl]
    X0=row["X0"];Y0=row["Y0"];Z0=row["Z0"]-115.5
    X1=row["X1"];Y1=row["Y1"];Z1=row["Z1"]-115.5
    X2=row["X2"];Y2=row["Y2"];Z2=row["Z2"]-115.5
    X3=row["X3"];Y3=row["Y3"];Z3=row["Z3"]-115.5
    if Y0==0:
        ax1.plot([Z0,Z1],[X0,X1],"-",color=col)
        ax1.plot([Z1,Z2],[X1,X2],"-",color=col)
        ax1.plot([Z2,Z3],[X2,X3],"-",color=col)
ax1.grid()
ax1.set_xlim(-70,0)
ax1.set_ylim(-10,30)
ax1.set_xlabel("Z (mm)")
ax1.set_ylabel("X (mm)")
ax1.set_title("Dispersion of beam (Ronchi 400)",fontweight="bold",fontsize=20)
ax1.tick_params(axis='both', which='major', labelsize=20)
#ax1.tick_params(axis='both', which='minor', labelsize=8)


gs01 = gs0[1].subgridspec(1, 3)
ax2=fig.add_subplot(gs01[0,0])
ax3=fig.add_subplot(gs01[0,1])
ax4=fig.add_subplot(gs01[0,2])


for index, row in df_hoe.iterrows():
    wl=row["wave"]
    col=ColorDict[wl]
    X0=row["X0"];Y0=row["Y0"];Z0=row["Z0"]-115.835
    X1=row["X1"];Y1=row["Y1"];Z1=row["Z1"]-115.835
    X2=row["X2"];Y2=row["Y2"];Z2=row["Z2"]-115.835
    X3=row["X3"];Y3=row["Y3"];Z3=row["Z3"]-115.835
    if X0==0:
        ax2.plot([Z2,Z3],[Y2,Y3],"-",color=col)
ax2.grid()
ax2.set_xlim(-20,0)
ax2.set_ylim(0,30)
ax2.set_xlabel("Z (mm)")
ax2.set_ylabel("X (mm)")
ax2.set_title("HOE",fontweight="bold",fontsize=20)
ax2.tick_params(axis='both', which='major', labelsize=20)
#ax1.tick_params(axis='both', which='minor', labelsize=8)


for index, row in df_R400.iterrows():
    wl=row["wave"]
    col=ColorDict[wl]
    X0=row["X0"];Y0=row["Y0"];Z0=row["Z0"]-115.5
    X1=row["X1"];Y1=row["Y1"];Z1=row["Z1"]-115.5
    X2=row["X2"];Y2=row["Y2"];Z2=row["Z2"]-115.5
    X3=row["X3"];Y3=row["Y3"];Z3=row["Z3"]-115.5
    if Y0==0:
        ax3.plot([Z2,Z3],[X2,X3],"-",color=col)
ax3.grid()
ax3.set_xlim(-20,0)
ax3.set_ylim(0,30)
ax3.set_xlabel("Z (mm)")
ax3.set_ylabel("X (mm)")
ax3.set_title("Ronchi 400",fontweight="bold",fontsize=20)
ax3.tick_params(axis='both', which='major', labelsize=20)
#ax1.tick_params(axis='both', which='minor', labelsize=8)


for index, row in df_R200.iterrows():
    wl=row["wave"]
    col=ColorDict[wl]
    X0=row["X0"];Y0=row["Y0"];Z0=row["Z0"]-115.5
    X1=row["X1"];Y1=row["Y1"];Z1=row["Z1"]-115.5
    X2=row["X2"];Y2=row["Y2"];Z2=row["Z2"]-115.5
    X3=row["X3"];Y3=row["Y3"];Z3=row["Z3"]-115.5
    if Y0==0:
        ax4.plot([Z2,Z3],[X2,X3],"-",color=col)
ax4.grid()
ax4.set_xlim(-20,0)
ax4.set_ylim(0,30)
ax4.set_xlabel("Z (mm)")
ax4.set_ylabel("X (mm)")
ax4.set_title("Ronchi 200",fontweight="bold",fontsize=20)
ax4.tick_params(axis='both', which='major', labelsize=20)
#ax1.tick_params(axis='both', which='minor', labelsize=8)



plt.tight_layout()
plt.savefig("Beam4BeamDispersion.pdf")